In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim import Adam
import os 
import json


In [2]:
annotation_train_dir = "./ann_subsample/ann_subsample"

In [3]:
annotation_train_dir = "./ann_subsample/ann_subsample/"
files = os.listdir(annotation_train_dir)

poses = {}

for pose in files:
    with open(os.path.join(annotation_train_dir,pose)) as f:
        data = json.load(f)
        poses.update({pose.replace('.json',''): data})

In [4]:
data = {

}

for pose in poses.keys():
    data.update({pose:[]})


for pose in poses.keys():
    for image in poses[pose].keys():
        landmarks = poses[pose][image]["landmarks"]
        if len(landmarks[0]) == 21:
            data[pose].append(torch.tensor(landmarks[0]).squeeze().flatten())



In [5]:
for pose in data.keys():
    data.update({pose:torch.stack(data[pose])})



In [6]:
from typing import Dict

class LandmarkSet(Dataset):
    def __init__(self, data:Dict[str,torch.Tensor]):
        x = torch.tensor([])
        y = torch.tensor([])

        for index,pose in enumerate(data.keys()):
            x = torch.concat((x,data[pose]))
            _y = torch.zeros((18))
            _y[index] = 1
            y = torch.concat((y,_y.repeat((len(data[pose]),1))))
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index) :
        return self.x[index],self.y[index]
    


In [7]:
landmark_set = LandmarkSet(data)
length = len(landmark_set)
train_data,valid_data,test_data= random_split(landmark_set,[int(x * length) for x in [0.8,0.1,0.1]])# 


In [15]:
print(train_data[:1])

(tensor([[0.3839, 0.3899, 0.3723, 0.3799, 0.3677, 0.3629, 0.3666, 0.3477, 0.3621,
         0.3390, 0.3739, 0.3477, 0.3759, 0.3259, 0.3773, 0.3119, 0.3784, 0.3011,
         0.3833, 0.3479, 0.3846, 0.3248, 0.3856, 0.3091, 0.3861, 0.2960, 0.3924,
         0.3507, 0.3932, 0.3276, 0.3938, 0.3125, 0.3938, 0.3002, 0.4011, 0.3555,
         0.4019, 0.3370, 0.4016, 0.3249, 0.4011, 0.3145]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]))


In [8]:
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_dataloader = DataLoader(valid_data,batch_size=64,shuffle=True)

In [9]:

GestureModel = nn.Sequential(
    nn.Linear(42,84),
    nn.ReLU(),
    nn.Linear(84,168),
    nn.ReLU(),
    nn.Linear(168,84),
    nn.ReLU(),
    nn.Linear(84,42),
    nn.ReLU(),
    nn.Linear(42,21),
    nn.ReLU(),
    nn.Linear(21,18)
)

In [12]:
EPOCHS = 100
lr = 1e-2
loss_func = nn.CrossEntropyLoss()
opt = Adam(GestureModel.parameters(),lr=lr)
def fit():
    for epoch in range(EPOCHS):
        epoch_train_loss = 0
        epoch_valid_loss = 0
        for batch in train_dataloader:
            xb,yb = batch
            preds = GestureModel(xb)
            loss = loss_func(preds,yb)
            with torch.no_grad():
                epoch_train_loss += loss
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        print(f"Epoch {epoch} train Loss: {epoch_train_loss/len(train_dataloader)}")

        for batch in valid_dataloader:
            with torch.no_grad():
                xb,yb = batch
                preds = GestureModel(xb)
                loss = loss_func(preds,yb)
                epoch_valid_loss += loss
        
        print(f"Epoch {epoch} valid Loss: {epoch_valid_loss/len(valid_dataloader)}")



fit()




Epoch 0 train Loss: 2.8918802738189697
Epoch 0 valid Loss: 2.903540849685669
Epoch 1 train Loss: 2.890639066696167
Epoch 1 valid Loss: 2.9025185108184814
Epoch 2 train Loss: 2.8897578716278076
Epoch 2 valid Loss: 2.9003407955169678
Epoch 3 train Loss: 2.8897316455841064
Epoch 3 valid Loss: 2.90094256401062
Epoch 4 train Loss: 2.8897593021392822
Epoch 4 valid Loss: 2.9011247158050537
Epoch 5 train Loss: 2.8895468711853027
Epoch 5 valid Loss: 2.8988285064697266
Epoch 6 train Loss: 2.8901140689849854
Epoch 6 valid Loss: 2.8988962173461914
Epoch 7 train Loss: 2.8897018432617188
Epoch 7 valid Loss: 2.898339033126831
Epoch 8 train Loss: 2.8896284103393555
Epoch 8 valid Loss: 2.8986265659332275
Epoch 9 train Loss: 2.8894448280334473
Epoch 9 valid Loss: 2.9004180431365967
Epoch 10 train Loss: 2.8895857334136963
Epoch 10 valid Loss: 2.898347854614258
Epoch 11 train Loss: 2.889143943786621
Epoch 11 valid Loss: 2.8980767726898193
Epoch 12 train Loss: 2.8893232345581055
Epoch 12 valid Loss: 2.8979

In [156]:
import torch.nn as nn
 
ce = nn.CrossEntropyLoss()
 
logits = torch.tensor([[-1.90, -0.29, -2.30], [-0.29, -1.90, -2.30]])
target = torch.tensor([[0., 1., 0.], [1., 0., 0.]])
logits.shape
target.shape
# print("Cross entropy: %.3f" % ce(logits, target))

torch.Size([2, 3])